<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-08-19 17:56:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.2’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.05s   

2019-08-19 17:56:18 (392 KB/s) - ‘house-votes-84.data.2’ saved [18171/18171]



In [2]:
!head house-votes-84.data

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('house-votes-84.data', header=None
                 , names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [5]:
df = df.replace({"?" : np.NaN, "y" : 1, "n" : 0})
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [6]:
rep = df[df.party == 'republican']
dem = df[df.party == 'democrat']


import collections
from scipy.stats import ttest_1samp
import pprint

def bprint(*args, indent=4):
    pprint.pprint(args,indent=4)
    

# flattens nested dictionaries by making dict.subdict.somekey into a new key subdict__somekey under dict
# https://stackoverflow.com/questions/6027558/flatten-nested-dictionaries-compressing-keys
def flatten(d, parent_key='', sep='__'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
    
# force map to evaluate and put the result into a list
def Map(func, x):
    v = []
    for thing in map(func, x):
        v.append(thing)
    return v

# coerce to dict
def Dict(thing):
    # fix for named tuples
    if getattr(thing, "_asdict"):
        return dict(thing._asdict())
    else:
        return dict(thing)
     
    

In [7]:
# checking out the 1samp quickly
foobar = lambda df: df.drop('party',axis=1).apply(lambda column: ttest_1samp(column, 0, nan_policy='omit'))
Map(foobar, [rep, dem])

[handicapped-infants       (6.159569669016066, 5.434587970316366e-09)
 water-project             (12.28932045559371, 2.525482675130834e-24)
 budget                    (5.02528816355446, 1.3107521045196396e-06)
 physician-fee-freeze    (115.61141812122193, 4.438624448197153e-159)
 el-salvador-aid         (56.73182528352142, 1.0573299896233442e-109)
 religious-groups         (38.02862698794469, 1.4751069893318224e-83)
 anti-satellite-ban          (7.1448429295405, 2.954236755913103e-11)
 aid-to-contras           (5.305692786041199, 3.7903923869499887e-07)
 mx-missile                  (4.61979066121233, 7.73159348469265e-06)
 immigration              (14.376541013291384, 7.541248569126767e-31)
 synfuels                 (4.903414959073885, 2.3188606914425685e-06)
 education                 (32.24127789030702, 2.295255722861058e-70)
 right-to-sue            (31.153578519095596, 4.1784590643494416e-69)
 crime                    (91.79687721631203, 2.672177432250972e-140)
 duty-free          

In [8]:
from scipy.stats import ttest_ind

# ttest_ind now between the two dataframes
np_rep = rep.drop('party',axis=1)
np_dem = dem.drop('party',axis=1)
    
ttest_ind_results = Map(lambda column: {"column": column,
                                        "result": Dict(ttest_ind(np_rep[column], np_dem[column], nan_policy='omit')),
                                        "mean_rep": np_rep[column].mean(), 
                                        "mean_dem": np_dem[column].mean()}, 
                        np_rep.columns)
bprint("TTest_Ind", ttest_ind_results)

(   'TTest_Ind',
    [   {   'column': 'handicapped-infants',
            'mean_dem': 0.6046511627906976,
            'mean_rep': 0.18787878787878787,
            'result': {   'pvalue': 1.613440327937243e-18,
                          'statistic': -9.205264294809222}},
        {   'column': 'water-project',
            'mean_dem': 0.502092050209205,
            'mean_rep': 0.5067567567567568,
            'result': {   'pvalue': 0.9291556823993485,
                          'statistic': 0.08896538137868286}},
        {   'column': 'budget',
            'mean_dem': 0.8884615384615384,
            'mean_rep': 0.13414634146341464,
            'result': {   'pvalue': 2.0703402795404463e-77,
                          'statistic': -23.21277691701378}},
        {   'column': 'physician-fee-freeze',
            'mean_dem': 0.05405405405405406,
            'mean_rep': 0.9878787878787879,
            'result': {   'pvalue': 1.994262314074344e-177,
                          'statistic': 49.367081

In [9]:
def print_test_result(result_dict):
    print("Null Hypothesis:\n\tSupport among the two parties for {} is equal".format(result_dict['column']),
          "\nStatus:\t{}".format( "Reject" if result_dict['result']['pvalue'] < 0.05  else "Unable to Reject"),
          "\nDemocrat Mean Support: {}".format(result_dict['mean_dem']),
          "\nRepublican Mean Support: {}".format(result_dict['mean_rep']),
          "\n")

Map(lambda result_dict: print_test_result(result_dict), ttest_ind_results);

Null Hypothesis:
	Support among the two parties for handicapped-infants is equal 
Status:	Reject 
Democrat Mean Support: 0.6046511627906976 
Republican Mean Support: 0.18787878787878787 

Null Hypothesis:
	Support among the two parties for water-project is equal 
Status:	Unable to Reject 
Democrat Mean Support: 0.502092050209205 
Republican Mean Support: 0.5067567567567568 

Null Hypothesis:
	Support among the two parties for budget is equal 
Status:	Reject 
Democrat Mean Support: 0.8884615384615384 
Republican Mean Support: 0.13414634146341464 

Null Hypothesis:
	Support among the two parties for physician-fee-freeze is equal 
Status:	Reject 
Democrat Mean Support: 0.05405405405405406 
Republican Mean Support: 0.9878787878787879 

Null Hypothesis:
	Support among the two parties for el-salvador-aid is equal 
Status:	Reject 
Democrat Mean Support: 0.21568627450980393 
Republican Mean Support: 0.9515151515151515 

Null Hypothesis:
	Support among the two parties for religious-groups is eq

In [10]:
def to_dataframe(array_of_dict):
    fixed = Map(lambda d: flatten(d), array_of_dict)
    return pd.DataFrame(fixed)
test_df = to_dataframe(ttest_ind_results)
test_df

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working


,column,result__statistic,result__pvalue,mean_rep,mean_dem
0,handicapped-infants,-9.205264,1.613440e-18,0.187879,0.604651
1,water-project,0.088965,9.291557e-01,0.506757,0.502092
2,budget,-23.212777,2.070340e-77,0.134146,0.888462
3,physician-fee-freeze,49.367082,1.994262e-177,0.987879,0.054054
4,el-salvador-aid,21.136693,5.600520e-68,0.951515,0.215686
5,religious-groups,9.737576,2.393672e-20,0.897590,0.476744
6,anti-satellite-ban,-12.526188,8.521033e-31,0.240741,0.772201
7,aid-to-contras,-18.052093,2.824718e-54,0.152866,0.828897
8,mx-missile,-16.437503,5.030793e-47,0.115152,0.758065
9,immigration,1.735912,8.330248e-02,0.557576,0.471483


In [18]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

               


test_df['index'] = test_df.index
fig = go.Figure(data=go.Scatter(x=test_df['index'],y=test_df['result__statistic'], 
                                text=test_df['column'], mode='markers'))
fig.update_layout(title='Differences in Voting')
# todo label xaxes or label points and so forth
fig.show()


- Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

Handicapped infants.
- Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

Physician-fee-freeze
- Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference

Water-project. p-value is .929.
